In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy
from cdfvi import funcs

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda_v2 = ccl.Cosmology(
    Omega_c=0.94, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_w0wa = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0=-0.8, wa=-1.0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)


In [ ]:
sim_vals = dict(
    n_obj = 1200,
    z_lens_mean = 0.74,
    z_lens_scale = 0.49,
    z_source_mean = 2.31,
    z_source_scale = 0.45,
)

In [ ]:
sims = funcs.simulate_lens_systems(**sim_vals)
z_source_vals = sims.z_source.to_numpy()
z_lens_vals = sims.z_lens.to_numpy()

In [ ]:
vals = funcs.R_flat(cosmo, z_lens_vals, z_source_vals)
vals_no_lambda = funcs.R_flat(cosmo_no_lambda, z_lens_vals, z_source_vals)
vals_wowa = funcs.R_flat(cosmo_w0wa, z_lens_vals, z_source_vals)

err_level = 0.05
scatter = np.random.normal(0, err_level, size=len(z_source_vals))
errs = vals*err_level
errs_no_lambda = vals_no_lambda*err_level
errs_wowa = vals_wowa*err_level

In [ ]:
_ = plt.errorbar(z_lens_vals, vals, errs, marker='.', ls="")
_ = plt.xlabel(r"$z_{d}$")
_ = plt.ylabel(r"$R_{ds} \equiv \frac{\chi_{d}}{\chi_{s}}$")
_ = plt.xlim(0, 2.5)
_ = plt.ylim(0, 1.1)

In [ ]:
bin_edges = np.logspace(-0.10, 0.5, 10)
for bin_low, bin_hi in zip(bin_edges[0:-1], bin_edges[1:]):
    source_mask = np.bitwise_and(z_source_vals > bin_low, z_source_vals <= bin_hi)
    _ = plt.errorbar(
        z_lens_vals[source_mask],
        vals[source_mask], 
        errs[source_mask],
        marker='.',
        ls="", 
        label=f"z_s {bin_low:.2f} - {bin_hi:.2f}",
        alpha=0.4,
    )
_ = plt.xlabel(r"$z_{d}$")
_ = plt.ylabel(r"$R_{ds} \equiv \frac{\chi_{d}}{\chi_{s}}$")    
_ = plt.xlim(0, 2.5)
_ = plt.ylim(0, 1.1)
_ = plt.legend()